In [ ]:
import random
import copy

In [ ]:
!pip install  dgl -f https://data.dgl.ai/wheels/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 48.4 MB/s eta 0:00:00
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 41.7 MB/s eta 0:

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import dgl
import dgl.function as fn
from dgl import DGLGraph
!export DGLBACKEND pytorch

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
F1 = open("cora.cites", "r")
F1 = [i.replace("\n", " ").replace("\t", " ") for i in F1.readlines()]

F1 = [i.split() for i in F1]
F1 = [[int(j) for j in i] for i in  F1]

In [ ]:
F2 = open("cora.content", "r")
F2 = [i.replace("\n", " ").replace("\t"," ") for i in F2.readlines()]

F2 = [i.split() for i in F2]

for i in range(len(F2)):

    for j in range(len(F2[i])):

        try:

            F2[i][j] = int(F2[i][j])

        except ValueError:

            continue

for i in range(len(F2)):

    F2[i] = [F2[i][0], F2[i][1 : -1], F2[i][-1]]

Label_Dict = {}

count = 0

for i in range(len(F2)):

    label = F2[i][-1]

    if Label_Dict.get(label, None) == None:

        Label_Dict.update({label : count})
        count += 1

In [ ]:
Node_ID = {}
count = 0
for i in F2:

    if Node_ID.get(i[0], None) == None:

        Node_ID.update({i[0] : count})
        count += 1

F2 = [[Node_ID[i[0]], torch.tensor(i[1]), Label_Dict[i[2]]] for i in F2]

In [ ]:
#Generating CORA graph

cora_edges = [[Node_ID[i[1]] for i in F1] + [Node_ID[i[0]] for i in F1], [Node_ID[i[0]] for i in F1] + [Node_ID[i[1]] for i in F1]]
cora_edges = (torch.tensor(cora_edges[0]), torch.tensor(cora_edges[1]))

cora_graph = dgl.graph(cora_edges)

In [ ]:
Features = torch.zeros((cora_graph.nodes().size()[0], len(F2[0][1])))

Node_Labels = torch.zeros((cora_graph.nodes().size()[0], ))

for i in F2:

    Features[i[0], :] = i[1]

    Node_Labels[i[0]] = i[2]

cora_graph.ndata['h'] = Features
cora_graph.ndata['labels'] = Node_Labels

In [ ]:
Nodes = [i for i in range(cora_graph.nodes().size()[0])]
random.shuffle(Nodes)

Train = Nodes[0 : int(0.65 * len(Nodes))]
Val = Nodes[int(0.65 * len(Nodes)) : int(int(0.75 * len(Nodes)))]
Test = Nodes[int(0.75 * len(Nodes)) : ]

#Creating training, testing and validation masks
train_mask = torch.zeros((cora_graph.nodes().size()[0], ), dtype = torch.bool)
val_mask = torch.zeros((cora_graph.nodes().size()[0], ), dtype = torch.bool)
test_mask = torch.zeros((cora_graph.nodes().size()[0], ), dtype = torch.bool)

for i in Train:

    train_mask[i] = True

for i in Val:

    val_mask[i] = True

for i in Test:

    test_mask[i] = True

Label_Set = torch.zeros((cora_graph.nodes().size()[0], len(Label_Dict)))

for i in F2:

    node = i[0]
    label = i[2]
    Label_Set[node][label] = 1

In [ ]:
class Net(nn.Module):

    def __init__(self, n1, n2):

        super(Net, self).__init__()

        self.Layer1 = dgl.nn.GraphConv(n1, n2, norm = 'both', weight = True, bias = True, allow_zero_in_degree = True)

        self.Activ1 = nn.Hardswish()

        self.Layer2 = dgl.nn.GraphConv(n2, n2, norm = 'both', weight = True, bias = True, allow_zero_in_degree = True)

        self.Activ2 = nn.LeakyReLU()

        self.Layer3 = dgl.nn.GraphConv(n2, n2, norm = 'both', weight = True, bias = True, allow_zero_in_degree = True)

    def forward(self, g, x):

        x = self.Layer1(g, x)

        x = self.Activ1(x)

        x = self.Layer2(g, x)

        x = self.Activ2(x)

        x = self.Layer3(g, x)

        return x

In [ ]:
class NodeAdditionGNN(nn.Module):

    def __init__(self, n1, n2, n3):

        super(NodeAdditionGNN, self).__init__()

        self.Net = Net(n1, n2)

        self.Activ = nn.LeakyReLU()

        self.Layer = nn.Linear(n2, n3)

    def NodeCluster(self, g, x, y, embeddings, train, val, test, num_clusters):

        embeds = embeddings.detach().numpy()

        Clust = KMeans(n_clusters = num_clusters)

        Clust.fit(embeds)

        centres = Clust.cluster_centers_

        labels = Clust.labels_

        #So, clusters have been generated!

        new_x = torch.zeros((x.size()[0] + num_clusters, x.size()[1]), dtype = x.dtype)
        new_y = torch.zeros((y.size()[0] + num_clusters, y.size()[1]), dtype = y.dtype)
        new_embeds = torch.zeros((embeddings.size()[0] + num_clusters, embeddings.size()[1]), dtype = embeddings.dtype)

        new_train = torch.zeros((train.size()[0] + num_clusters, ), dtype = train.dtype)
        new_val = torch.zeros((val.size()[0] + num_clusters, ), dtype = val.dtype)
        new_test = torch.zeros((test.size()[0] + num_clusters, ), dtype = test.dtype)

        new_train[0 : train.size()[0]] = train
        new_val[0 : val.size()[0]] = val
        new_test[0 : test.size()[0]] = test

        num_nodes = g.number_of_nodes()

        g.add_nodes(num_clusters)

        cluster_count = [0 for i in range(num_clusters)]

        for i in range(len(labels)):

            g.add_edges(i, labels[i] + num_nodes)
            g.add_edges(labels[i] + num_nodes, i)

            cluster_count[labels[i]] += 1

            new_x[labels[i] + num_nodes, :] += x[i, :]
            new_embeds[labels[i] + num_nodes, :] += embeddings[i, :]

        print("Cluster count: ", cluster_count)

        new_x[0 : x.size()[0], :] = x
        new_embeds[0 : embeddings.size()[0], :] = embeddings
        new_y[0 : y.size()[0], :] = y

        for i in range(num_clusters):

            new_x[i + num_nodes, :] /= max(cluster_count[i], 1)
            new_embeds[i + num_nodes, :] /= max(cluster_count[i], 1)

        return g, new_x, new_y, new_embeds, new_train, new_val, new_test

    def forward(self, g, x):

        #Type 1 node additions:
        #Based on initial node features

        embeds = self.Net(g, x)

        x = self.Activ(embeds)

        x = self.Layer(x)

        return x, embeds

    def evaluate(self, g, x):

        with torch.no_grad():

            pred = self.Net(g, x)

            pred = self.Activ(pred)

            pred = self.Layer(pred)

            return pred

In [ ]:
def Train(g, x, y, train, val, test, epochs, lr, num_clusters):

    g.ndata['h'] = x

    Model = NodeAdditionGNN(g.ndata['h'].size()[1], 400, y.size()[1])

    optim = torch.optim.Adam(Model.parameters(), lr = lr, weight_decay = 1.e-5)

    LossFunc = nn.CrossEntropyLoss(reduction = 'mean')

    ProbabLayer = nn.Softmax(dim = 1)

    embed = copy.deepcopy(x)

    train_acc = []
    val_acc = []

    for i in range(epochs):

        Model.train()

        optim.zero_grad()

        g.ndata['h'] = x

        y_pred, embeds = Model(g, x)

        train_loss = LossFunc(y_pred[train], y[train])

        train_loss.backward()

        optim.step()

        Model.eval()

        val_loss = LossFunc(y_pred[val], y[val])

        # with torch.no_grad():

        #     Model.NodeCluster(g, x, y, x, train, val, test, 10)

        train_acc.append(train_loss.item())
        val_acc.append(val_loss.item())

        print("Epoch : " + str(i) + ", Train Loss = " + str(train_loss.item()) + ", Validation Loss = " + str(val_loss.item()))

        #Node clustering

        if i%10 == 0:

            with torch.no_grad():

                g, x, y, embeds, train, val, test = Model.NodeCluster(g, x, y, embeds, train, val, test, num_clusters)

        else:

            continue

    Model.eval()

    with torch.no_grad():

        y_pred = Model.evaluate(g, x)

        test_loss = LossFunc(y_pred[test], y[test])

        print("Test Loss = " + str(test_loss.item()))

    #Computing accuracy of results

    P = nn.Softmax(dim = 1)

    acc = 0.

    with torch.no_grad():

        y_pred = P(Model.evaluate(g, x))

        y_pred = y_pred[test]
        y_test = y[test]

        for i in range(y_test.size()[0]):

            if torch.argmax(y_pred[i, :]) == torch.argmax(y_test[i, :]):

                acc += 1

        print("Accuracy = ", acc/y_test.size()[0])

    #Plotting model accuracy

    # plt.figure(figsize = (8, 6), dpi = 300)
    # plt.plot([i for i in range(len(train_acc))], train_acc, 'ro--', linewidth = 3, markersize = 8, label = 'Training Loss')
    # plt.plot([i for i in range(len(val_acc))], val_acc, 'go--', linewidth = 3, markersize = 8, label = 'Validation Loss')
    # plt.title('Loss accross epochs: Periodic Node addition', fontsize = 20)
    # plt.xlabel('Epochs', fontsize = 18)
    # plt.ylabel('Cross-Entropy Loss', fontsize = 18)
    # plt.xticks(fontsize = 15)
    # plt.yticks(fontsize = 15)
    # plt.legend(fontsize = 16)
    # plt.savefig('EpochLoss.jpeg', bbox_inches = 'tight')
    # plt.show()

    return Model, g, x, y, train, val, test, test, acc

In [ ]:
Acc = []

for i in range(10):

    Acc.append(Train(copy.deepcopy(cora_graph), copy.deepcopy(Features), copy.deepcopy(Label_Set), copy.deepcopy(train_mask), copy.deepcopy(val_mask), copy.deepcopy(test_mask), 39, 1.e-2, 25)[-1])

Epoch : 0, Train Loss = 1.9439281225204468, Validation Loss = 1.9438061714172363


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster count:  [49, 71, 229, 24, 50, 228, 37, 269, 124, 73, 46, 22, 114, 33, 23, 19, 16, 5, 214, 642, 167, 104, 71, 48, 30]
Epoch : 1, Train Loss = 1.7910010814666748, Validation Loss = 1.8202861547470093
Epoch : 2, Train Loss = 1.6865317821502686, Validation Loss = 1.6827086210250854
Epoch : 3, Train Loss = 1.4055513143539429, Validation Loss = 1.4381921291351318
Epoch : 4, Train Loss = 1.2787275314331055, Validation Loss = 1.2781299352645874
Epoch : 5, Train Loss = 1.0462019443511963, Validation Loss = 1.061163306236267
Epoch : 6, Train Loss = 0.8870057463645935, Validation Loss = 0.9401536583900452
Epoch : 7, Train Loss = 0.8387605547904968, Validation Loss = 0.9134250283241272
Epoch : 8, Train Loss = 0.7872097492218018, Validation Loss = 0.8568186163902283
Epoch : 9, Train Loss = 0.6876754760742188, Validation Loss = 0.7138069868087769
Epoch : 10, Train Loss = 0.7096099257469177, Validation Loss = 0.7078709602355957


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster count:  [202, 53, 137, 109, 464, 3, 39, 3, 195, 18, 100, 1, 140, 79, 3, 183, 48, 3, 186, 1, 166, 13, 33, 251, 303]
Epoch : 11, Train Loss = 0.6808212995529175, Validation Loss = 0.6939188838005066
Epoch : 12, Train Loss = 0.622255802154541, Validation Loss = 0.6911909580230713
Epoch : 13, Train Loss = 0.5872137546539307, Validation Loss = 0.6881024241447449
Epoch : 14, Train Loss = 0.5538472533226013, Validation Loss = 0.6551966071128845
Epoch : 15, Train Loss = 0.5213759541511536, Validation Loss = 0.6206222772598267
Epoch : 16, Train Loss = 0.5009557008743286, Validation Loss = 0.6024993062019348
Epoch : 17, Train Loss = 0.48151707649230957, Validation Loss = 0.58895343542099
Epoch : 18, Train Loss = 0.4727270305156708, Validation Loss = 0.606791615486145
Epoch : 19, Train Loss = 0.44655999541282654, Validation Loss = 0.6093922853469849
Epoch : 20, Train Loss = 0.4181146025657654, Validation Loss = 0.598541796207428


...

Cluster count:  [166, 68, 81, 144, 57, 51, 4, 181, 358, 2, 3, 304, 1, 16, 226, 6, 71, 166, 1, 29, 1, 109, 445, 19, 224]
Epoch : 11, Train Loss = 0.7580884695053101, Validation Loss = 0.7614999413490295
Epoch : 12, Train Loss = 0.7057763934135437, Validation Loss = 0.7361006736755371
Epoch : 13, Train Loss = 0.6649291515350342, Validation Loss = 0.7134048342704773
Epoch : 14, Train Loss = 0.631929874420166, Validation Loss = 0.7056888937950134
Epoch : 15, Train Loss = 0.5413851141929626, Validation Loss = 0.6431983709335327
Epoch : 16, Train Loss = 0.5185473561286926, Validation Loss = 0.6410608887672424
Epoch : 17, Train Loss = 0.5195632576942444, Validation Loss = 0.6431452035903931
Epoch : 18, Train Loss = 0.5017081499099731, Validation Loss = 0.6233706474304199
Epoch : 19, Train Loss = 0.47347262501716614, Validation Loss = 0.5937854051589966
Epoch : 20, Train Loss = 0.4576454758644104, Validation Loss = 0.589073121547699


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster count:  [143, 271, 54, 6, 181, 105, 15, 7, 103, 194, 5, 93, 351, 2, 45, 553, 129, 35, 159, 93, 4, 5, 139, 63, 3]
Epoch : 21, Train Loss = 0.49323901534080505, Validation Loss = 0.6061156988143921
Epoch : 22, Train Loss = 0.4818354845046997, Validation Loss = 0.5984203815460205
Epoch : 23, Train Loss = 0.4892441928386688, Validation Loss = 0.6126025915145874
Epoch : 24, Train Loss = 0.4647846817970276, Validation Loss = 0.593850314617157
Epoch : 25, Train Loss = 0.4454593062400818, Validation Loss = 0.5803354382514954
Epoch : 26, Train Loss = 0.43971186876296997, Validation Loss = 0.5797128677368164
Epoch : 27, Train Loss = 0.4291550815105438, Validation Loss = 0.5828313231468201
Epoch : 28, Train Loss = 0.41928035020828247, Validation Loss = 0.5954682230949402
Epoch : 29, Train Loss = 0.4118483364582062, Validation Loss = 0.6018467545509338
Epoch : 30, Train Loss = 0.4033869206905365, Validation Loss = 0.5999693870544434


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster count:  [18, 289, 632, 150, 109, 4, 64, 205, 6, 8, 4, 53, 3, 20, 101, 158, 31, 4, 396, 153, 92, 7, 101, 3, 172]
Epoch : 31, Train Loss = 0.424563467502594, Validation Loss = 0.597525954246521
Epoch : 32, Train Loss = 0.4216022789478302, Validation Loss = 0.58519446849823
Epoch : 33, Train Loss = 0.4119909405708313, Validation Loss = 0.5742555856704712
Epoch : 34, Train Loss = 0.3989022970199585, Validation Loss = 0.5844696164131165
Epoch : 35, Train Loss = 0.3876633942127228, Validation Loss = 0.586432933807373
Epoch : 36, Train Loss = 0.38183876872062683, Validation Loss = 0.5887063145637512
Epoch : 37, Train Loss = 0.3740394711494446, Validation Loss = 0.5964651107788086
Epoch : 38, Train Loss = 0.36762484908103943, Validation Loss = 0.5722037553787231
Test Loss = 0.7243016362190247
Accuracy =  0.843426883308715


In [ ]:
temp = test_mask.long()

temp = torch.sum(temp).item()

print(temp)

677


In [ ]:
BAcc = [i/677 for i in Acc]

mean = sum(BAcc)/len(BAcc)

var = sum([i ** 2 for i in BAcc])/len(BAcc) - mean ** 2

print(mean, var ** 0.5)

0.8255539143279172 0.013086348925724968
